You will probably not need this notebook!! Maybe for later .... I just leave it here in case...

In [ ]:
%pylab ipympl
from nugridpy import ppn 
from nugridpy import utils
import pickle
import sys, operator, glob, os

In [ ]:
pwd

In [ ]:
data_dir = '/data/nugrid_data/projects/tina/iprocess-impact-MC/'
casename = 'default'

In [ ]:
import jbutils # this is a local module in this directyory

# data_dir needs to be the same in `jbutils.py` as above!

In [ ]:
# these are the available pre-computed one-zone ppn runs
os.listdir(data_dir)

Select neutron densities to include in analysis:

In [ ]:
ydenn=arange(8,16.1,0.5) # this is the range of constant neutron densities 
                        # used in the multiple ppn runs
ppn_cases = []
for yd in ydenn:
    denn="%7.1e"%(10**yd)
    ppn_cases.append('run_'+casename+'_denn'+str(denn))

# check that list of cases actually exist in data_dir
for case in ppn_cases:
    if os.path.isdir(data_dir+case): 
        print(f"{case} is in list")
    else:
        print(f"{case} dir does not exist")
        print("change `these_ppn_cases` list")
        

In [ ]:
# %%capture
# create dictionary only first time
# to add new cases comment
pa_cases = {} 

# create a ppn.abu_vector instance for each case
for case in ppn_cases:
    pa=ppn.abu_vector(os.path.join(data_dir+"one-zone-run_default",case))
    pa_cases[case] = pa

In [ ]:
# Test: check what cases are loaded
pa_cases.keys()

In [ ]:
# check individual ppn cases
case = list(pa_cases.keys())[-1]
# case = 'run_default_denn1.0e+15'
print(case)

In [ ]:
# Test: last cycle of case
cycle = pa_cases[case].get('mod')[-1]    # this is the last cycle
print(case,cycle)

In [ ]:
%%capture --no-display

# Test: iso_abundance plot

size=3;ifig=110
m1,m2 = 132,170; y1,y2 = -7, -2.2
close(ifig);figure(ifig,figsize=(2*size,size))
pa_cases[case].iso_abund(cycle, decayed=False, amass_range=(m1,m2),ylim=[y1,y2],point_set=0)
text(0.9*m2,y2-0.5,"$\mathrm{2^{nd}}$ peak, undecayed")
tight_layout()

In [ ]:
case

In [ ]:
# Test: create x-time instance
px=ppn.xtime(data_dir+'one-zone-run_default/'+case)

In [ ]:
# Test: time evolution plot to check if equilibrium has been reached
# doplot = False
doplot = True
if doplot:
    close(114);figure(114)
    logx=True
    xthing = 'time'
    #xthing = 'cycle'
    if xthing == 'cycle': logx=False
    things = ['FE 56','FE 57','NI 66','Y  91','Y  92','Y  93','Y  94','Y  95','Y  96',\
              'BA138','BA140','BA142','BA144','BA146','PB208']
    for i,thing in enumerate(things):
        px.plot(xthing,thing,shape=utils.linestylecb(i)[1]+utils.linestylecb(i)[0],logy=True,logx=logx,\
                markevery=utils.linestyle(i,55,5)[1],legend=thing,show=False)
    ylabel('mass fraction');xlabel("log t/[yr]")
    ylim(-13,-1.7); xlim(-6,-1.5); title("")
    #ylim(-13,-1.7); xlim(0,4.5); title("")
    legend(loc=3,fontsize=8)

In [ ]:
ppn_cases

In [ ]:
%%capture
# while doing an elemental abundance plot wrt solar abundance, 
# record scaled elemental abundance and neutron densities for further processing
# elemental abundance plots are saved to png for the records

el_abu_log_dict={}; Nn={}  # wither initialize for first time
                            # or load previous from Nn_el_abu_unsorted.pkl

cases = ppn_cases[-1:] # we will load later precomputed files, so maybe only do one case here
cases = ppn_cases # we will load later precomputed files, so maybe only do one case here
print(cases)
sol_ab = data_dir+'one-zone-run_default/'+'../data_utils/iniab1.4E-02As09.ppn'
for i,case in enumerate(ppn_cases):
    close(100+i);figure(100+i)
    print (i,case)
    cycle = pa_cases[case].get('mod')[-1]                                                 # this is the last cycle
    pa_cases[case].elemental_abund(cycle,zrange=[25,85],ref_filename=sol_ab,ylim=[-2,8])  # elements only returned that are in zrange
    el_abu_log_dict[case] = pa_cases[case].el_abu_log
    Nn[case]              = pa_cases[case].get('densn',cycle)
    title("log Neutron density = %4.1f"%log10(Nn[case]))
    cc="%4.1f"%log10(Nn[case])
    cc=cc.strip().zfill(4)

In [ ]:
pa_elname = pa.el_name

In [ ]:
# write Nn_el_abu_unsorted.pkl only of you have done new runs
write_Nn_unsorted = False 
if write_Nn_unsorted:
    afile = open('Nn_el_abu_unsorted_'+casename+'.pkl', 'wb')
    pickle.dump([el_abu_log_dict,Nn,pa.el_name], afile)
    afile.close()

In [ ]:
### load Nn_el_abu_unsorted.pkl
load_Nn_unsorted = False 
if load_Nn_unsorted:
    afile = open('Nn_el_abu_unsorted_'+casename+'.pkl','rb')
    el_abu_log_dict,Nn,pa_elname = pickle.load(afile)
    afile.close()

In [ ]:
# At this point we have two dictionaries, Nn and el_abu_log_dict that do not 
# have to be sorted by Nn, maybe because you have added cases later, so let's
# sort keys by Nn values:

# sort, create elem_dict and write if sort_Nn = True
sort_Nn = True 
if sort_Nn:
    Nnsorted = sorted(Nn.items(), key=operator.itemgetter(1))
    cases_sorted = [Nnsorted[i][0] for i in range(len(Nnsorted))]
    Nnsorted = {}
    for case in cases_sorted: Nnsorted[case] = Nn[case]
    #print(cases_sorted)

    # create a elemental abundance dictionary in which for each available element 
    # the log elem abundance as extracted during elemental_abund plotting are available

    elem_dict = {} #always start with a new dictionary

    for element in pa.el_name:
    #    elem_dict[element] = array([el_abu_log_dict[case][pa_cases[case].el_name.index(element)] for case in ppn_cases]
        elem_dict[element] = array([el_abu_log_dict[case][pa.el_name.index(element)] for case in cases_sorted])

    # write elem_dict dictionary into python pickle
    afile = open('Nn_elem_dict_'+casename+'.pkl', 'wb')
    pickle.dump([Nnsorted,elem_dict], afile)
    afile.close()

Now, maybe you want to create a few runs with different nuclear physics assumptions. Create another `elem_dict` dictionary and write it into a python pickle file as well.

Then you can come back and read these for furhter analysis.

Or you come here for the first time, and want to just read and plot pre-calculated data:

In [ ]:
read_another_Nn_elem = False
# read Nn_elem_dict.pkl for plotting
if read_another_Nn_elem:
    afile = open('Nn_elem_dict_'+casename+'.pkl','rb') 
    Nn,elem_dict = pickle.load(afile)
    afile.close()

In [ ]:
read_another_Nn_elem = False
# read other Nn_elem_dict.pkl for plotting, for examle a case with CS137(n,g) up
if read_another_Nn_elem:
    afile = open('Nn_elem_dict_'+casename+'.pkl','rb')
    Nn_CS137up,elem_dict_CS137up = pickle.load(afile)
    afile.close()

## Snake plot
Three- or four-element plots are especially useful to compare the different predictions of different neutron density predictions with elemental ratios. 

### Network predictions
We use the elemental abundances extracted for these cases, maybe several sets, to make the _snake_ plot. This is a plot where we have for each neutron density one dot in a three- or four-element plot.

In [ ]:
# set elements to plot: 
# [els[0], els[1], els[2], els[3]] 
# [els[0]/els[1]] on x axis [els[2]/els[3]] on y axis
els = ['Ba', 'La', 'La','Eu']
# els = ['Ce', 'Nd', 'Sm','Eu']

# to set Nn colorbar range find 
logNn = log10(array(list(Nn.values())))
min_,max_ = logNn.min(),logNn.max()

size = 6
close(74);figure(74,figsize=(1.6*size,size))
scatter(elem_dict[els[0]]-elem_dict[els[1]],elem_dict[els[2]]-elem_dict[els[3]],\
        c=logNn,cmap='jet')
plot(elem_dict[els[0]]-elem_dict[els[1]],elem_dict[els[2]]-elem_dict[els[3]],'--',\
     color='grey',label='default')


jbutils.plot_elemratios(els=els,selections = ['S', 'I', 'R1', 'R2'], uls=False)        

clim(min_, max_) # set the min/max of the colorbar 
cbar = colorbar()
cbar.set_label('$\mathsf{N_n}$', rotation=90)


# legend()
xlim(-0.8,1.),ylim(-.6,1.3)
xlabel('['+els[0]+'/'+els[1]+']');ylabel('['+els[2]+'/'+els[3]+']')
tight_layout()
